****机器学习降维的三种常见的方法-使用PCA，LDA，tSNE对MNIST数据集进行降维分析****


* 主成分分析（PCA）-无监督线性方法

* 线性判别分析（LDA）-监督线性方法

* t-分布随机邻域嵌入（t-sne）-非线性概率方法

In [ ]:
#导入数据处理包
import numpy as np
import pandas as pd
#导入绘图包
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
#导入sklearn工具
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

**MNIST说明**：mnist集合由42000行和785列组成。有28 x 28像素的数字图像（共有784列）以及一个额外的标签列，这本质上是一个类标签，用于说明行对每个数字的贡献是1还是9。每行组件包含一个介于1和0之间的值，这描述了每个像素的强度。

In [ ]:
#读取数据集
train = pd.read_csv('../input/train.csv')
train.head()

In [ ]:
#抽取真实值梭所在的列
target = train['label']
#将真实值所在的列删除后作为训练数据
train = train.drop('label',axis=1)

***1. 主成分分析 Principal Component Analysis (PCA)***<br>
PCA是一种线性转换算法，旨在将数据的原始特征投影到一组较小的特征（或子空间），同时仍保留大部分信息。为此，算法试图找到最合适的方向/角度（主要成分），以最大化新子空间的方差。我们必须理解，主成分是相互正交的（直角思考）。因此，当在我们的新子空间中生成协方差矩阵（测量两个变量之间的关联程度）时，协方差矩阵的非对角值将为零，只有对角（或特征值）将为非零。正是这些对角线值代表了我们正在讨论的主要组件的方差，或者是关于我们的特性的可变性的信息。因此，当主成分分析试图最大化这一方差时，该方法试图找到包含数据点或信息（方差）相对于所有数据点的最大排列/子集的方向（主成分）。

**计算数据集的协方差矩阵的特征值和特征向量**

In [ ]:
#标准化数据集
from sklearn.preprocessing import StandardScaler
X = train.values
X_std = StandardScaler().fit_transform(X)

#计算特征向量与特征值之间的协方差矩阵
mean_vec = np.mean(X_std,axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#创建一个（特征向量，特征值）的元组
eig_pairs = [(np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
#对每对特征向量，特征值从高到低排序
eig_pairs.sort(key=lambda x:x[0],reverse=True)
#根据特征值计算解释性方差，方差值越大，说明特征越重要
tot = sum(eig_vals)
#单个解释性方差
var_exp = [(i/tot)*100 for i in sorted(eig_vals,reverse=True)]
#累计解释性方差
cum_vaar_exp = np.cumsum(var_exp)

**绘图显示**

In [ ]:
trace1 = go.Scatter(
    x=list(range(784)),
    y = cum_vaar_exp,
    mode = 'lines+markers',
    name = "'Cumulative Explained Variance'",
    line = dict(
        shape='spline',
        color='goldenrod'))
trace2 = go.Scatter(
    x=list(range(784)),
    y = var_exp,
    mode='lines+markers',
    name="'Individual Explained Variance'",
    line=dict(
        shape='linear',
        color='black'))
fig = tls.make_subplots(insets=[{'cell':(1,1),'l':0.7,'b':0.5}],print_grid=True)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,1)
fig.layout.title='Explained Variance plots - Full and Zoomed-in'
fig.layout.xaxis=dict(range=[0,80],title='Feature columns')
fig.layout.yaxis=dict(range=[0,60],title='Explained Variance')
py.iplot(fig,filename='styled-scatter')

上面两个图：一个较小的图嵌入到较大的图中。较小的图（绿色和红色）显示了个体的分布，并解释了所有特征值的差异，而较大的图（金色和黑色）描述了解释性差异的放大部分。

正如我们所看到的，在我们的784个特性或列中，大约90%的解释方差可以通过使用超过200个特性来描述。因此，如果要在这方面实现PCA，提取前200个特性将是一个非常合理的选择，因为它们已经占据了大部分数据。

In [ ]:
#sklearn的PCA方法
n_components = 30
pca = PCA(n_components=n_components).fit(train.values)
#抽取PCA的组件值-特征值
eigenvalues = pca.components_.reshape(n_components,28,28)
eigenvalues = pca.components_

***特征值可视化***<br>
正如上面提到的，由于PCA方法寻求获得捕获最大方差（分散数据点最多）的最佳方向（或特征向量）。因此，可视化这些方向及其相关特征值可能是有益的（也是很酷的）。为了本笔记本的目的和速度，我将调用PCA从数据集中仅提取前30个特征值，并将前5个特征值与其他一些较小的特征值进行视觉比较

In [ ]:
n_row = 4
n_col = 7

plt.figure(figsize=(13,12))
for i in list(range(n_row*n_col)):
    offset = 0
    plt.subplot(n_row,n_col,i+1)
    plt.imshow(eigenvalues[i].reshape(28,28),cmap='jet')
    title_text = 'Eigenvalue'+str(i+1)
    plt.title(title_text,size=6.5)
    plt.xticks(())
    plt.yticks(())
plt.show()

上面的子图描绘了PCA方法决定为我们的数字数据集生成的前30个最佳方向或主成分分量。有趣的是，当将第一个分量“特征值1”与第28个分量“特征值28”进行比较时，很明显在搜索过程中生成了更复杂的方向或分量，以最大化新特征子空间中的方差。

***mnist数字集可视化***<br>
现在，为了有趣和好奇，让我们绘制实际的mnist数字集，看看底层数据集实际代表什么，而不是仅仅关注1和0。

In [ ]:
plt.figure(figsize=(14,12))
for digit_num in range(0,70):
    plt.subplot(7,10,digit_num+1)
    grid_data = train.iloc[digit_num].as_matrix().reshape(28,28)
    plt.imshow(grid_data,interpolation='none',cmap='afmhot')
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()

**使用sklearn工具包进行PCA分析**

In [ ]:
#删除前面创建的变量
del X

X = train[:6000].values

#删除前面创建的变量
del train

#标准化数据
X_std = StandardScaler().fit_transform(X)
#使用5个组件进行PCA降维
pca = PCA(n_components=5)
pca.fit(X_std)
X_5d = pca.transform(X_std)

Target = target[:6000]

上面的代码首先使用了sklearn的StandardScaler标准化数据集<br>
接下来，我们调用sklearn的内置pca函数，在其参数n_components中提供希望将数据投影到的组件/维度的数量。例如在我们解释性方差图中，我们将通过观察捕获的方差与每个特征的特征值的比例来选择激励组件。<br>
最后，我调用fit和transform方法，将PCA模型与标准化的数字数据集相匹配，然后通过在数据上应用维数约简进行转换。<br>

**PCA表示的交互可视化:**<br>
当涉及到这些降维方法时，散点图是最常见的实现方法，因为它们允许对聚类（如果存在）进行非常方便的可视化，这正是我们在绘制前两个主要成分时要做的事情，如下所示：

In [ ]:
trace0 = go.Scatter(
    x = X_5d[:,0],
    y = X_5d[:,1],
    mode = 'markers',
    text = Target,
    showlegend=False,
    marker=dict(
        size = 8,
        color=Target,
        colorscale='jet',
        showscale=False,
        line = dict(
            width = 2,
            color='rgb(255,255,255)'),
        opacity = 0.8))

data = [trace0]

layout = go.Layout(
    title='Principal Component Analysize(PCA)',
    hovermode='closest',
    xaxis=dict(
        title='First Principal Component',
        ticklen=5,
        zeroline=False,
        gridwidth=2,),
    yaxis=dict(
        title='Second Principal Component',
        ticklen=5,
        gridwidth=2,),
    showlegend=True)

fig = dict(data=data,layout=layout)
py.iplot(fig,filename='styled-scatter')

正如上图所示，从散点图中大致可以看出几个簇。这些集群代表了每个数据点应该贡献的底层数字，因此你可能会认为实现和可视化PCA非常容易。然而，问题在于Python实现的微小细节，因为正如前面提到的，PCA实际上是一个不受监督的方法，它不依赖于类标签。在生成散点图时，会产生成串的颜色

***k-Means聚类***<br>
因为PCA是一种无监督的方法，因此我们假设MNIST数据集没有类标签，那么我们应该如何在新的特征空间中分离这些数据点？这个时候我们可以使用K-Means聚类算法得到不同的聚类簇，通过簇我们可以知道数据底层分类中所包含的一些重要的信息。
试想一下，我们没有得到这个数字集的类标签，因为毕竟PCA是一个无监督的方法。因此，我们如何能够在新的特性空间中分离数据点？我们可以在新的PCA投影数据上应用一种聚类算法，并希望得到不同的聚类，这将告诉我们一些关于数据中底层类分离的信息。

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=9)
x_clustered = kmeans.fit_predict(X_5d)

trace_Kmeans = go.Scatter(x=X_5d[:,0],y=X_5d[:,1],mode='markers',
                         showlegend=False,
                         marker=dict(
                             size=8,
                             color=x_clustered,
                             colorscale='Portland',
                             showscale=False,
                             line=dict(
                                 width=2,
                                 color='rgb(255,255,255)'
                             )))
layout = go.Layout(
    title='KMeans Clustering',
    hovermode='closest',
    xaxis=dict(
        title='First Principal Compoonent',
        ticklen=5,
        zeroline=False,
        gridwidth=2,),
    yaxis=dict(
        title='Second Principal Component',
        ticklen=5,
        gridwidth=2,
    ),
    showlegend=True)
data = [trace_Kmeans]
fig1 = dict(data=data,layout=layout)
py.iplot(fig1,filename='svm')

从上图看，与单纯地在PCA预测中添加类标签相比，由kmeans算法生成的集群似乎提供了更清晰的集群划分。这并不奇怪，因为PCA是一种无监督的方法，因此不适合分离不同的类标签。

2. 线性判别分析 Linear Discriminant Analysis (LDA)<br>
与PCA相似，LDA也是一种常用于降维任务的线性变换方法。然而，与无监督学习算法不同，LDA属于有监督学习方法。因此，LDA的目标是利用关于类标签的可用信息，通过计算组件轴（线性判别法），使不同类之间的分离最大化。

LDA的目标是在保留类分离信息的同时仍然减少数据集的维度。分为下面这四个步骤：
1. 预计方式 Projected Means<br>
由于这种方法的设计考虑了类标签，因此我们首先需要建立一个合适的度量标准，用它来测量不同类之间的“距离”。假设我们有一组属于一个特定类W的数据点X。因此，在LDA中，第一步是将这些点投影到一个新行Y上，该行通过转换包含特定于类的信息。有了这一点，我们的想法是找到一些方法，最大限度地分离这些新的预测变量。为此，我们首先计算投影平均值。
2. 散射矩阵及其解 Scatter Matrices and their solutions<br>
引入了我们的投影平均值，现在我们需要找到一个函数来表示平均值之间的差异，然后将其最大化。就像线性回归一样，最基本的情况是找到最适合的线，我们需要在这个上下文中找到方差的等价物。因此，这就是我们引入散射矩阵的地方，散射等于方差。
3. 选择最佳投影矩阵 Selecting Optimal Projection Matrices<br>
4. 将特征转换为新的子空间 Selecting Optimal Projection Matrices<br>

In [ ]:
from IPython.display import display,Math,Latex

In [ ]:
lda = LDA(n_components=5)
X_LDA_2D = lda.fit_transform(X_std,Target.values)

In [ ]:
traceLDA = go.Scatter(
    x = X_LDA_2D[:,0],
    y = X_LDA_2D[:,1],
    mode = 'markers',
    text = Target,
    showlegend=True,
    marker = dict(
        size = 8,
        color = Target,
        colorscale = 'Jet',
        showscale = False,
        line = dict(
            width=2,
            color='rgb(255,255,255)'),
    opacity=0.8))

data = [traceLDA]

layout = go.Layout(
    title="Linear Discrimiant Analysis(LDA)",
    hovermode = 'closest',
    xaxis=dict(
        title='First Linear Discriminant',
        ticklen=5,
        zeroline=False,
        gridwidth=2,),
    yaxis=dict(
        title='Second Linear Discrimiant',
        ticklen=5,
        gridwidth=2,),
    showlegend=False)

fig = dict(data=data,layout=layout)
py.iplot(fig,filename='style-scatter')

从上面的散点图可以看出，与使用类标签实现PCA相比，使用LDA时，数据点更加清晰地聚集在一起。这是使用类标签来监督方法的固有优势。简而言之，为正确的工作选择正确的工具。

3. ***t-sne（t-分布随机邻域嵌入） T-SNE ( t-Distributed Stochastic Neighbour Embedding )***<br>
自2008年van der Maaten和 Hinton提出T-SNE方法以来，T-SNE方法一直广受欢迎。与前面讨论的PCA和LDA的两种线性方法不同，T-SNE是一种非线性概率降维方法。



因此，T-SNE算法的内部机制不是着眼于最大化信息或类分离的方向/轴，而是将点之间的欧几里德距离转换为条件概率。然后将student-t分布应用于这些概率，这些概率用作度量，以计算一个数据点与另一个数据点之间的相似性。



然而，本摘要对Maaten和Hinton的原始T-SNE论文没有任何形式的证明，[因此请在此处查看原始的T-SNE论文](http://www.cs.toronto.edu/~hinton/absps/tsne.pdf)。

In [ ]:
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(X_std)

通过简单地调用了t-sne算法，我们将数字数据拟合到模型中，并通过fit_转换减少其维数。最后，让我们在散点图中绘制新特征空间中的前两个组件。

In [ ]:
traceTSNE = go.Scatter(
    x = tsne_results[:,0],
    y = tsne_results[:,1],
    mode='markers',
    showlegend=True,
    marker=dict(
        size = 8,
        color = Target,
        colorscale='Jet',
        line = dict(
            width = 2,
            color = 'rgb(255,255,255)'),
        opacity=0.8))

data = [traceTSNE]

layout = dict(title="TSNE(T-Distributed Stochastic Neighbour Embedding)",
             hovermode = 'closest',
             yaxis = dict(zeroline=False),
             xaxis = dict(zeroline=False),
             showlegend=False)

fig = dict(data=data,layout=layout)
py.iplot(fig,filename='style-scatter')

从T-SNE散点图来看，首先要注意的是，簇（甚至子簇）被很好地定义和分离，从而产生了杰克逊·波洛克式的现代艺术视觉效果，甚至比PCA和LDA方法更为明显。这种提供非常好的集群可视化的能力可以归结为算法的拓扑保留属性。


然而，T-SNE也有其缺点。当算法识别簇/子簇时，可能会出现多个局部极小值，这可以从散点图中得到，在散点图中我们可以看到相同颜色的簇作为2个子簇存在于图的不同区域中。